# Cleaning the weather dataset

## Importing data and modules

In [1]:
import pandas as pd

weather = pd.read_csv('../data/weather.csv')

## Checking for null values, data shape

In [2]:
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [3]:
weather.shape

(2944, 22)

In [4]:
weather.isnull().sum().sum()

0

In [5]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


The only non object features are `Station`, `Tmax`, `Tmin`, `Dewpoint`, `ResultSpeed`, and `ResultDir`. The built in pandas methods for checking for missing data should suffice to sort any missing values for them. For the remaining features we have to look more closely. 

In [6]:
weather.Station.value_counts()

1    1472
2    1472
Name: Station, dtype: int64

In [7]:
weather.Tmax.isnull().sum()

0

In [8]:
weather.Tmin.isnull().sum()

0

In [9]:
weather.DewPoint.isnull().sum()

0

In [10]:
weather.ResultSpeed.isnull().sum()

0

In [11]:
weather.ResultDir.isnull().sum()

0

## Checking for missing data

**So there's no true null values in the columns that are already coded as numeric, but we might need to look more carefully at the remaining ones. According to the data dictionary, most features encode missing values with an 'M'.**

### Date

The date column is in a consistent format (yyyy-mm-dd) that we can use for merging throughout the weather dataset.

In [12]:
weather.Date.describe()

count           2944
unique          1472
top       2010-05-25
freq               2
Name: Date, dtype: object

In [13]:
weather.Date.map(len).value_counts()

10    2944
Name: Date, dtype: int64

### Tavg

In [14]:
Tavg_missing = weather['Tavg'].map(lambda val: val == 'M')
Tavg_missing.sum()

11

There are very few missing values for average temperature; let's drop those rows:

In [15]:
weather = weather[~Tavg_missing]

In [16]:
weather['Tavg'] = weather['Tavg'].astype(float)

In [17]:
Depart_missing = weather['Depart'].map(lambda val: val == 'M')
Depart_missing.sum()

1461

In [18]:
weather.drop('Depart', 1, inplace=True)

### WetBulb

In [19]:
WetBulb_missing = weather['WetBulb'].map(lambda val: val == 'M')
WetBulb_missing.sum()

4

In [20]:
weather = weather[~WetBulb_missing]

In [21]:
weather['WetBulb'] = weather['WetBulb'].astype(float)

### Heat, Cool

In [22]:
Heat_missing = weather['Heat'].map(lambda val: val == 'M')
Heat_missing.sum()

0

In [23]:
weather['Heat'] = weather['Heat'].astype(float)

In [24]:
Cool_missing = weather['Cool'].map(lambda val: val == 'M')
Cool_missing.sum()

0

In [25]:
weather['Cool'] = weather['Cool'].astype(float)

### Sunrise, Sunset

In [26]:
Sunrise_missing = weather['Sunrise'].map(lambda val: val == 'M')
Sunrise_missing.sum()

0

In [27]:
weather['Sunrise'].value_counts().head()

-       1460
0416     104
0417      64
0419      40
0420      32
Name: Sunrise, dtype: int64

In [28]:
Sunset_missing = weather['Sunset'].map(lambda val: val == 'M')
Sunset_missing.sum()

0

In [29]:
weather['Sunset'].value_counts().head()

-       1460
1931      95
1930      56
1929      48
1927      32
Name: Sunset, dtype: int64

### CodeSum, Depth

In [30]:
CodeSum_missing = weather['CodeSum'].map(lambda val: val == 'M')
CodeSum_missing.sum()

0

In [31]:
codesums  = set(weather.CodeSum.str.split(' ').map(tuple))

codes = set()
for cs in codesums:
    for code in cs:
        codes.add(code)
codes.discard('')
codes = list(codes)


In [32]:
def build_df_codesum(row, codes):
    cs = set(row['CodeSum'].split())
    code_out = [int(code in cs) for code in codes]
    code_out = pd.Series(data=code_out, index=codes)
    return code_out

weather[codes] = weather.apply(lambda row:build_df_codesum(row, codes), 1)

In [33]:
weather.drop('CodeSum', 1, inplace=True)

In [34]:
Depth_missing = weather['Depth'].map(lambda val: val == 'M')
Depth_missing.sum()

1460

In [35]:
weather['Depth'].value_counts()

0    1469
M    1460
Name: Depth, dtype: int64

In [36]:
weather.drop('Depth', 1, inplace=True)

### Water1, SnowFall, PrecipTotal, 

In [37]:
Water1_missing = weather['Water1'].map(lambda val: val == 'M')
Water1_missing.sum()

2929

In [38]:
weather['Water1'].value_counts()

M    2929
Name: Water1, dtype: int64

In [39]:
weather.drop('Water1', 1, inplace=True)

It's very unlikely that snow would be falling during the time period (summer months) when the weather data was collected; we can see that there were only 13 instances of any evidence of snowfall. Let's drop the snowfall column.

In [40]:
SnowFall_missing = weather['SnowFall'].map(lambda val: val == 'M')
SnowFall_missing.sum()

1460

In [41]:
weather['SnowFall'].value_counts()

M      1460
0.0    1456
  T      12
0.1       1
Name: SnowFall, dtype: int64

In [42]:
weather.drop('SnowFall', 1, inplace=True)

In [46]:
PrecipTotal_missing = weather['PrecipTotal'].map(lambda val: val == 'M')
PrecipTotal_missing.sum()

2

In [47]:
weather = weather[~PrecipTotal_missing]

In [48]:
weather.PrecipTotal = weather.PrecipTotal.map(lambda x: x.replace('T', '0.001')).astype(float)

# StnPressure


In [49]:
StnPressure_missing = weather['StnPressure'].map(lambda val: val == 'M')
StnPressure_missing.sum()

2

In [50]:
weather = weather[~StnPressure_missing]  
weather['StnPressure'] = weather['StnPressure'].map(float)

## SeaLevel

In [51]:
SeaLevel_missing = weather['SeaLevel'].map(lambda val: val == 'M')
SeaLevel_missing.sum()

7

In [52]:
weather = weather[~SeaLevel_missing]  
weather['SeaLevel'] = weather['SeaLevel'].map(float)

## AvgSpeed

In [53]:
AvgSpeed_missing = weather['AvgSpeed'].map(lambda val: val == 'M')
AvgSpeed_missing.sum()

0

In [54]:
weather['AvgSpeed'] = weather['AvgSpeed'].map(float)

## Exporting to csv

In [55]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,...,RA,DZ,GR,MIFG,SQ,FG,TS,FG+,VCTS,FU
0,1,2007-05-01,83,50,67.0,51,56.0,0.0,2.0,0448,...,0,0,0,0,0,0,0,0,0,0
1,2,2007-05-01,84,52,68.0,51,57.0,0.0,3.0,-,...,0,0,0,0,0,0,0,0,0,0
2,1,2007-05-02,59,42,51.0,42,47.0,14.0,0.0,0447,...,0,0,0,0,0,0,0,0,0,0
3,2,2007-05-02,60,43,52.0,42,47.0,13.0,0.0,-,...,0,0,0,0,0,0,0,0,0,0
4,1,2007-05-03,66,46,56.0,40,48.0,9.0,0.0,0446,...,0,0,0,0,0,0,0,0,0,0


In [56]:
weather.isnull().sum().sum()

0

In [57]:
weather.to_csv('../data/weather_cleaned.csv', index=False)